# Natočení vrutů stejným směrem

### Načtení potřebných knihoven

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import shutil
import cv2 as cv
from tqdm.auto import tqdm
import time
import PIL
from PIL import Image
import pathlib
import pandas as pd
import itertools

### Natočení vrutů

In [2]:
def ds_prep(ds_path,version):
    
    os.chdir(ds_path)
    ds_split_names = ['train', 'valid', 'test']
    org_ds_path = r"C:\Users\HP\Documents\GIT-BP\screw_orig"
    categories = os.listdir(r"C:\Users\HP\Documents\GIT-BP\screw_orig\test")
    new_ds_path = "Screw_ds-v" + str(version) + "(oriented)"
    
    if not os.path.exists(new_ds_path):
        os.makedirs(new_ds_path)
        os.chdir(new_ds_path)
        new_ds_dir=os.getcwd()
        print("New dataset folder made. \n \n" + "Dataset directory:\t" + new_ds_dir + " \t( " + str(len(categories)) + " categories )")
        
        for i in ds_split_names:
            if not os.path.exists(i):
                os.makedirs(i)
                os.chdir(i)
                for j in categories:
                    os.makedirs(j)
            os.chdir(new_ds_dir)
    else:
        print("Use new version number.")
    
    
    src_orig = org_ds_path + '\\test'
    dst_new = new_ds_dir + "\\All_images"
    shutil.copytree(src_orig, dst_new)
    
    for k in categories:
        mask_origin = dst_new + "\\" + k
        src_img_list = os.listdir(mask_origin)
        mask_src = mask_origin + "\\" + src_img_list[0]
        mask_dst = mask_origin + "\\0_MASK.png"
        os.rename(mask_src,mask_dst)
    
    train_good_imgs_dir = r"C:\Users\HP\Documents\GIT-BP\screw_orig\train\good"
    train_good_imgs_list = os.listdir(r"C:\Users\HP\Documents\GIT-BP\screw_orig\train\good")
    for n in train_good_imgs_list:
        shutil.copy(train_good_imgs_dir + "\\" + n,
                    dst_new + "\\good" +"\\0" + n)
    
    
    for k in tqdm(categories[1:],desc = "Matching all category templates "):
        
        cv_mask_0 = cv.imread(dst_new + "\\" + categories[0] + "\\0_MASK.png")
        org_mask_path = dst_new + "\\" + k + "\\0_MASK.png"
        org_mask_copy_path = dst_new + "\\" + k + "\\0_MASK(copy).png"
        
        shutil.copy(org_mask_path,
                    org_mask_copy_path)
        
        cv_nrot_mask_copy = cv.imread(org_mask_copy_path)
        
        m_h, m_w = cv_nrot_mask_copy.shape[:2]
        m_center = (m_w/2, m_h/2)
                
        mask_Value = list()
        mask_Phi = list()
        
        for mask_phi in range(0,360):
            mask_rot_mtx = cv.getRotationMatrix2D(center=m_center, angle=mask_phi, scale=1)
            rot_mask_copy = cv.warpAffine(src=cv_nrot_mask_copy,
                                    M=mask_rot_mtx,
                                    dsize=(m_w, m_h),
                                    borderMode=cv.BORDER_REPLICATE)
            rot_mask_match = cv.matchTemplate(cv_mask_0,rot_mask_copy,cv.TM_SQDIFF_NORMED)
            min_mask_val, max_val, min_loc, max_loc = cv.minMaxLoc(rot_mask_match)
            mask_Value.append(min_mask_val)
            mask_Phi.append(mask_phi)
        
        mask_Phi_index = mask_Value.index(min(mask_Value))
        nrot_mask_cv = cv.imread(org_mask_path)
        f_rot_mtx = cv.getRotationMatrix2D(center=m_center, angle=mask_Phi[mask_Phi_index], scale=1)
        f_rot_mask = cv.warpAffine(src=nrot_mask_cv,
                                   M=f_rot_mtx,
                                   dsize=(m_w, m_h),
                                   borderMode=cv.BORDER_REPLICATE)
        f_img = cv.imwrite(org_mask_path,f_rot_mask)
        os.remove(org_mask_copy_path)
    time.sleep(0.1)
    
    for k in tqdm(categories, desc = "Matching images in each category "):
        
        nrot_img_path = dst_new + "\\" + k 
        nrot_img_dirs = os.listdir(nrot_img_path)
        list_of_nrot_images = nrot_img_dirs[::-1]
        
        for i in tqdm(range(1,len(list_of_nrot_images)),desc = f" Matching category : {k} "):
            
            img_mask = cv.imread(nrot_img_path + "\\0_MASK.png")
            
            org_img_path = nrot_img_path + "\\" + str(list_of_nrot_images[i])
            org_img_copy_path = nrot_img_path + "\\copy_" + str(list_of_nrot_images[i])
            
            shutil.copy(org_img_path,
                        org_img_copy_path)
            Value = list()
            Phi = list()
            
            
            cv_org_img_copy = cv.imread(org_img_copy_path)
            i_h, i_w = cv_org_img_copy.shape[:2]
            i_center = (i_w/2, i_h/2)
            
            for phi in range(0,360):
                img_rot_mtx = cv.getRotationMatrix2D(center=i_center, angle=phi, scale=1)
                rot_img_copy = cv.warpAffine(src=cv_org_img_copy,
                                              M=img_rot_mtx,
                                              dsize=(i_w, i_h),
                                              borderMode=cv.BORDER_REPLICATE)
                rot_img_match = cv.matchTemplate(img_mask,rot_img_copy,cv.TM_SQDIFF_NORMED)
                min_val, max_val, min_loc, max_loc = cv.minMaxLoc(rot_img_match)
                Value.append(min_val)
                Phi.append(phi)
                
            Phi_index = Value.index(min(Value))
            org_img_cv = cv.imread(org_img_path)
            f_rot_mtx = cv.getRotationMatrix2D(center=i_center, angle=Phi[Phi_index], scale=1)
            f_rot_img = cv.warpAffine(src=org_img_cv,
                                      M=f_rot_mtx,
                                      dsize=(i_w, i_h),
                                      borderMode=cv.BORDER_REPLICATE)
            f_img = cv.imwrite(org_img_path,f_rot_img)
            os.remove(org_img_copy_path)
        time.sleep(0.1)

os.chdir(r'\Users\HP\Documents\GIT-BP')

In [ ]:
ds_path = r"C:\Users\HP\Documents\GIT-BP"

h = ds_prep(ds_path,1.0)

### Rozdělení datasetu pro trénování, validaci a testování

In [ ]:
def ds_distribution(ds_version,distribution):
    ds_path_ = r"C:\Users\HP\Documents\GIT-BP\Screw_ds-v" + str(version) + "(oriented)"
    ds_path_oriented = ds_path_ + "\\All_images" 
    category_list = os.listdir(ds_path_oriented)
    [ a, b, c] = distribution
    ds_split_names = ['train', 'valid', 'test']
    dst_ = ds_path_ + "\\"
    dst_train = dst_ + ds_split_names[0]
    dst_valid = dst_ + ds_split_names[1]
    dst_test = dst_ + ds_split_names[2]
    for i in tqdm(category_list):
        cat = ds_path_oriented + "\\" + i
        img_names = os.listdir(cat)
        num_of_items = len(img_names)
        a_items = int(num_of_items*(distribution[0]/100))
        b_items = int(num_of_items*(distribution[1]/100))
        c_items = int(num_of_items*(distribution[2]/100))
        for a in img_names[:a_items]:
            shutil.copy(cat + "\\" + a,
                        dst_train + "\\" + i + "\\" + a)
        for b in img_names[a_items:(a_items+b_items)]:
            shutil.copy(cat + "\\" + b,
                        dst_valid + "\\" + i + "\\" + b)
        for c in img_names[(a_items+b_items):]:
            shutil.copy(cat + "\\" + c,
                        dst_test + "\\" + i + "\\" + c)
    time.sleep(0.1)

In [ ]:
ds_version = 1.0 
distribution = [50, 30, 20]

f = ds_distribution(ds_version,distribution)